In [2]:
%load_ext autoreload
%autoreload 2

repbrep = '../../'
outdir = '../../figures/'
results_dir = '../../results'

## Accuracy Plots

In [ ]:
from plotting import plot_classification_accuracies, plot_segmentation_accuracies
import pandas as pd
import altair_saver
import os
import numpy as np

# Segmentation Plots
f360_new = pd.read_parquet(os.path.join(repbrep,'results/f360_new.parquet'))
f360_new.dataset = 'Fusion360Seg'
mfcad_new = pd.read_parquet(os.path.join(repbrep,'results/mfcad_new.parquet'))
segmentation_predictions = pd.read_parquet(os.path.join(repbrep, 'results/segmentation_predictions.parquet'))
segmentation_predictions = pd.concat([segmentation_predictions,f360_new,mfcad_new],axis=0, ignore_index=True)
segmentation_predictions = segmentation_predictions[segmentation_predictions.model != 'Ours']
segmentation_predictions.loc[segmentation_predictions.model == 'NewModel', 'model'] = 'Ours'
f360_accuracy_plot = plot_segmentation_accuracies(segmentation_predictions,  'Fusion360Seg', 'macro', title='Fusion 360 Segmentation')
mfcad_accuracy_plot = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'macro', title='MFCAD Segmentation')

# Classification Plots
classification_predictions = pd.read_parquet(os.path.join(repbrep, 'results/classification_predictions.parquet'))
fabwave_new = pd.read_parquet(os.path.join(repbrep,'results/fabwave_new.parquet'))
cpreds_idx = classification_predictions[
    (classification_predictions.model == 'Ours') & 
    (classification_predictions.seed == 0) & 
    (classification_predictions.train_fraction == 1.0)]
cpreds_size = classification_predictions[
    (classification_predictions.model == 'Ours') & 
    (classification_predictions.seed == 0) & 
    (classification_predictions.test_idx == 0)]
ommitted_dict = dict(zip(cpreds_idx.test_idx, cpreds_idx.ommitted))
ts_dict = dict(zip(cpreds_size.train_fraction.values, cpreds_size.train_size.values))
get_ts = np.vectorize(lambda x: ts_dict[x])
get_ommitted = np.vectorize(lambda x : ommitted_dict[x])
fabwave_new['train_size'] = get_ts(fabwave_new.train_fraction)
fabwave_new['ommitted'] = get_ommitted(fabwave_new.test_idx)
classification_predictions = pd.concat([classification_predictions,fabwave_new],axis=0, ignore_index=True)

cp = classification_predictions.copy()
cp = cp[cp.model != 'Ours']
cp.model = np.vectorize(lambda x: 'UV-Net' if x == 'UV-Net' else 'Ours')(cp.model)

cp = cp[(cp.ommitted == False)]
cp = cp[(cp.label != 14) & (cp.label != 22)]

plot_classification_accuracies(cp, 'FabWave', title='FabWave Classification')#, size_proto_model='NewModel')
fabwave_accuracy_plot = plot_classification_accuracies(cp, 'FabWave', title='FabWave Classification')

os.makedirs(outdir, exist_ok=True)
altair_saver.save(f360_accuracy_plot, os.path.join(outdir, 'f360-accuracy-plot.pdf'))
altair_saver.save(mfcad_accuracy_plot, os.path.join(outdir, 'mfcad-accuracy-plot.pdf'))
altair_saver.save(fabwave_accuracy_plot, os.path.join(outdir, 'fabwave-accuracy-plot.pdf'))

## Dataset Gallery

In [ ]:
import torch
import numpy as np
from rendering import find_best_angle_from_part, render_part, render_mesh
from tqdm import tqdm
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
from automate import Part
from train_latent_space import BRepFaceAutoencoder
from matplotlib import pyplot as plt
import os


datasets = os.path.join(repbrep, 'datasets')
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
results_dir = os.path.join(repbrep, 'results')

f360seg_index_path = os.path.join(datasets, 'fusion360seg.json')
f360seg_zip_path = os.path.join(datasets, 'fusion360seg.zip')
model_checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')
computed_f360seg_codes_path = os.path.join(model_dir, 'fusion360seg_coded.pt')
render_losses_path = os.path.join(results_dir, 'f360_render_test_losses.pt')
figure_out_path = os.path.join(outdir, 'datasetgallery.png')

train_poses_path = os.path.join(datasets, 'f360seg_test_poses.npy')
train_zooms_path = os.path.join(datasets, 'f360seg_test_zooms.npy')

rows = 4
cols = 6
size = 5

grid_density = 100
seed = 42


with open(f360seg_index_path,'r') as f:
    index = json.load(f)
data =  ZipFile(f360seg_zip_path,'r')
parts_list = [index['template'].format(*x) for x in index['test']]

np.random.seed(42)
to_render_idx = np.random.choice(np.arange(len(parts_list)), rows*cols, replace=False)

gts = []

poses = np.load(train_poses_path)
zooms = np.load(train_zooms_path)



for k in tqdm(range(rows*cols)):
    i = to_render_idx[k]
    path = parts_list[k]
    part = Part(data.open(path).read().decode('utf-8'))
    pose = poses[i]
    zoom = zooms[i]
    ground_truth = render_part(part, pose, zoom)
    gts.append(ground_truth)

M = rows*cols
s = size
fig, axes = plt.subplots(int(M/cols), cols, figsize=(cols*s, s*int(M/cols)),gridspec_kw = {'wspace':0, 'hspace':0}, dpi=300)
for i in range(M):
    row = int(i / cols)
    col = i % cols
    axes[row,col].imshow(gts[i])
    axes[row,col].axis('off')
fig.savefig(figure_out_path)

In [ ]:
# New Model Version
import torch
import numpy as np
from rendering import find_best_angle_from_part, render_part, render_mesh
from tqdm import tqdm
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
from automate import Part
from train_latent_space import BRepFaceAutoencoder
from matplotlib import pyplot as plt
import os


datasets = os.path.join(repbrep, 'datasets')
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
results_dir = os.path.join(repbrep, 'results')

f360seg_index_path = os.path.join(datasets, 'fusion360seg.json')
f360seg_zip_path = os.path.join(datasets, 'fusion360seg.zip')
model_checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')
computed_f360seg_codes_path = os.path.join(model_dir, 'fusion360seg_coded.pt')
render_losses_path = os.path.join(results_dir, 'f360_render_test_losses.pt')
figure_out_path = os.path.join(outdir, 'datasetgallery.png')

train_poses_path = os.path.join(datasets, 'f360seg_test_poses.npy')
train_zooms_path = os.path.join(datasets, 'f360seg_test_zooms.npy')

rows = 4
cols = 6
size = 5

grid_density = 100
seed = 42


with open(f360seg_index_path,'r') as f:
    index = json.load(f)
data =  ZipFile(f360seg_zip_path,'r')
parts_list = [index['template'].format(*x) for x in index['test']]

np.random.seed(42)
to_render_idx = np.random.choice(np.arange(len(parts_list)), rows*cols, replace=False)

gts = []

poses = np.load(train_poses_path)
zooms = np.load(train_zooms_path)



for k in tqdm(range(rows*cols)):
    i = to_render_idx[k]
    path = parts_list[k]
    part = Part(data.open(path).read().decode('utf-8'))
    pose = poses[i]
    zoom = zooms[i]
    ground_truth = render_part(part, pose, zoom)
    gts.append(ground_truth)

M = rows*cols
s = size
fig, axes = plt.subplots(int(M/cols), cols, figsize=(cols*s, s*int(M/cols)),gridspec_kw = {'wspace':0, 'hspace':0}, dpi=300)
for i in range(M):
    row = int(i / cols)
    col = i % cols
    axes[row,col].imshow(gts[i])
    axes[row,col].axis('off')
fig.savefig(figure_out_path)

In [ ]:
len(index['train'])+len(index['test'])

## Reconstruction Gallery

In [ ]:
import torch
import numpy as np
from hybridbrep import get_camera_angle, get_norm_factors, render_segmented_mesh, RendererParams, grid_images
#from rendering import find_best_angle_from_part, render_part, render_mesh
from tqdm import tqdm
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
#from automate import Part
from hybridbrep import GeneralConvEncDec, HPart, HybridPartDataset
from automate import Part, PartOptions
#from train_latent_space import BRepFaceAutoencoder
from matplotlib import pyplot as plt
import os
import pandas as pd
from PIL import Image

datasets = os.path.join(repbrep, 'datasets')
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
results_dir = os.path.join(repbrep, 'results')

figure_out_path = os.path.join(outdir, 'reconstructions.png')


rows = 4
cols = 6
size = 5
N = 100

imsize=800

grid_density = 100

force_rendering = True

print('Loading Model')
ckpt_path = '/home/ben/Documents/research/repbrep/training_logs/reconstruction/new_with_edges/version_1/checkpoints/epoch=183-val_loss=0.002646.ckpt'
ckpt = torch.load(ckpt_path)
model = GeneralConvEncDec(64, 1024, 4)
model.load_state_dict(ckpt['state_dict'])

ablations_path = '../../results/recon_ablations.parquet'
ablations = pd.read_parquet(ablations_path)
errors = ablations[ablations.model == 'New Network, New Data']

print('Loading Index')
index_path = '../../datasets/fusion360seg.json'
data_path = '../../datasets/fusion360seg.zip'
with open(index_path, 'r') as f:
    index = json.load(f)

print('Counting Faces')
ds_test = HybridPartDataset('../../datasets/fusion360seg.json', '../../datasets/fusion360seg_hpart_fixed.zip', mode='test')
num_faces = {i:ds_test[i].faces.shape[0] for i in range(len(ds_test))}

print('Selecting Parts')
errors['faces'] = np.vectorize(lambda x: num_faces[x])(errors.test_idx)
errors['total_value'] = errors.value * errors.faces
total_errors = errors.groupby('test_idx').agg({'value':sum, 'total_value':sum}).reset_index()
total_errors['faces'] = np.vectorize(lambda x: num_faces[x])(total_errors.test_idx)
# Selection Criteria: lowest overall amount of loss with at least 20 faces

to_render_idx = total_errors[total_errors.faces > 20].sort_values('total_value',ascending=True).test_idx.values[:rows*cols]

print('Rendering')
gts = []
renders = []
with ZipFile(data_path, 'r') as zf:    
    for idx in tqdm(to_render_idx):
        key = index['template'].format(*index['test'][idx])
        with zf.open(key, 'r') as f:
            part_data = f.read().decode('utf-8')
        data = ds_test[idx]
        with torch.no_grad():
            pred = model.grid_enc_dec(data, N)
        V, F, C = preds_to_mesh(pred[0], N)
        opts = PartOptions()
        opts.set_quality = True
        opts.quality = 0.001
        part = Part(part_data, opts)

        color_pallet = plt.get_cmap('tab20')(np.arange(num_faces[idx]))[:,:3]

        camera_params = get_camera_angle(
            part.mesh.V, part.mesh.F, part.mesh_topology.face_to_topology,
            optimize='seg'
        )

        gt_im = render_segmented_mesh(
            part.mesh.V, part.mesh.F, part.mesh_topology.face_to_topology, color_pallet,
            camera_params=camera_params,
            render_params=RendererParams(imsize,imsize)
            )

        norm_center, norm_scale = get_norm_factors(data.V.numpy())
        
        recon_im = render_segmented_mesh(
            V, F, C, color_pallet, camera_params=camera_params,
            norm_center=norm_center,
            norm_scale=norm_scale,
            render_params=RendererParams(imsize,imsize)
        )

        gts.append(gt_im)
        renders.append(recon_im)

image_rows = []
for r in range(rows):
    curr_row = []
    for c in range(cols):
        idx = r*cols+c
        gt = gts[idx]
        recon = renders[idx]
        curr_row.append(gt)
        curr_row.append(recon)
    curr_row = np.stack(curr_row)
    image_rows.append(curr_row)
image_rows = np.stack(image_rows)

image_grid = grid_images(image_rows)

image = Image.fromarray(image_grid.astype(np.uint8))

image.save(figure_out_path)

## Greyscale Reconstruction for Overview

In [ ]:
# Version Using New Rendering Code
# Problems:
#  - normalizing parts and reconstructions separately makes them different sizes
#    - solution: have a non-normalized version of camera param finding

import torch
from tqdm import tqdm
from train_latent_space import BRepFaceAutoencoder
from rendering import render_segmented_mesh, get_camera_angle, grid_images, RendererParams
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
from automate import Part, PartOptions
import numpy as np
import os
from PIL import Image

datasets = os.path.join(repbrep, 'datasets')
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')

f360seg_index_path = os.path.join(datasets, 'fusion360seg.json')
f360seg_zip_path = os.path.join(datasets, 'fusion360seg.zip')
model_checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')
computed_f360seg_codes_path = os.path.join(model_dir, 'fusion360seg_coded.pt')
render_losses_path = os.path.join(results_dir, 'f360_render_test_losses.pt')
figure_out_path = os.path.join(outdir,'greyscale_reconstructions.png')

grid_density = 100 # Point Sampling Density Per Face

rows = 4
cols = 6
imsize=800

print('Loading and Filtering Losses')
testing_losses = torch.load(render_losses_path)
avg_losses = np.array([x[0].item() for x in testing_losses])
part_sizes = np.array([x[1] for x in testing_losses])
total_losses = avg_losses * part_sizes
avg_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][0])
total_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][2])
total_filtered = [x for x in total_sorted if x[1][1] > 20]

print('Loading Dataset')
with open(f360seg_index_path,'r') as f:
    index = json.load(f)
data =  ZipFile(f360seg_zip_path,'r')
parts_list = [index['template'].format(*x) for x in index['test']]

print('Loading Model')
model = BRepFaceAutoencoder(64,1024,4)
ckpt = torch.load(model_checkpoint_path)
model.load_state_dict(ckpt['state_dict'])

def predict(face_codes, model, N=grid_density):
    n_faces = face_codes.shape[0]
    line = torch.linspace(-0.1,1.1,N)
    grid = torch.cartesian_prod(line, line)
    grids = grid.repeat(n_faces,1)
    indices = torch.arange(n_faces).repeat_interleave(N*N, dim=0)
    with torch.no_grad():
        indexed_codes = face_codes[indices]
        uv_codes = torch.cat([grids, indexed_codes],dim=1)
        preds = model.decoder(uv_codes)
    return preds

codes = torch.load(computed_f360seg_codes_path)

gts = []
renders = []

renderer = None

options = PartOptions()
options.set_quality = True
options.quality = 0.001

for k in tqdm(range(rows*cols), 'Predicting and Rendering'):
    i = total_filtered[k][0]
    N = grid_density
    preds = predict(codes[parts_list[i]]['x'], model, N)
    V, F, C = preds_to_mesh(preds, N)
    part = Part(data.open(parts_list[i]).read().decode('utf-8'), options)
    
    camera_params = get_camera_angle(
        part.mesh.V, 
        part.mesh.F, 
        part.mesh_topology.face_to_topology,
        optimize='seg'
    )

    n_faces = part.mesh_topology.face_to_topology.max() + 1

    face_colors = np.ones((n_faces,3)).astype(int)*150

    gt_im = render_segmented_mesh(
        part.mesh.V, part.mesh.F, part.mesh_topology.face_to_topology, face_colors,
        camera_params=camera_params,
        render_params=RendererParams(imsize,imsize)
        )
    
    recon_im = render_segmented_mesh(
        V, F, C, face_colors, camera_params=camera_params,
        render_params=RendererParams(imsize,imsize)
    )

    gts.append(gt_im)
    renders.append(recon_im)

image_rows = []
for r in range(rows):
    curr_row = []
    for c in range(cols):
        idx = r*cols+c
        gt = gts[idx]
        recon = renders[idx]
        curr_row.append(gt)
        curr_row.append(recon)
    curr_row = np.stack(curr_row)
    image_rows.append(curr_row)
image_rows = np.stack(image_rows)

image_grid = grid_images(image_rows)

Image.fromarray(image_grid.astype(np.uint8)).save(figure_out_path)

In [38]:
import torch
import numpy as np
from hybridbrep import get_camera_angle, get_norm_factors, render_segmented_mesh, RendererParams, grid_images
#from rendering import find_best_angle_from_part, render_part, render_mesh
from tqdm import tqdm
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
#from automate import Part
from hybridbrep import GeneralConvEncDec, HPart, HybridPartDataset
from automate import Part, PartOptions
#from train_latent_space import BRepFaceAutoencoder
from matplotlib import pyplot as plt
import os
import pandas as pd
from PIL import Image

datasets = os.path.join(repbrep, 'datasets')
results_dir = os.path.join(repbrep, 'results')

figure_out_path = os.path.join(outdir,'greyscale_reconstructions.png')


rows = 4
cols = 6
size = 5
N = 100

imsize=800

grid_density = 100

force_rendering = True

print('Loading Model')
ckpt_path = '/home/ben/Documents/research/repbrep/training_logs/reconstruction/new_with_edges/version_1/checkpoints/epoch=183-val_loss=0.002646.ckpt'
ckpt = torch.load(ckpt_path)
model = GeneralConvEncDec(64, 1024, 4)
model.load_state_dict(ckpt['state_dict'])

ablations_path = '../../results/recon_ablations.parquet'
ablations = pd.read_parquet(ablations_path)
errors = ablations[ablations.model == 'New Network, New Data']

print('Loading Index')
index_path = '../../datasets/fusion360seg.json'
data_path = '../../datasets/fusion360seg.zip'
with open(index_path, 'r') as f:
    index = json.load(f)

print('Counting Faces')
ds_test = HybridPartDataset('../../datasets/fusion360seg.json', '../../datasets/fusion360seg_hpart_fixed.zip', mode='test')
num_faces = {i:ds_test[i].faces.shape[0] for i in range(len(ds_test))}

print('Selecting Parts')
errors['faces'] = np.vectorize(lambda x: num_faces[x])(errors.test_idx)
errors['total_value'] = errors.value * errors.faces
total_errors = errors.groupby('test_idx').agg({'value':sum, 'total_value':sum}).reset_index()
total_errors['faces'] = np.vectorize(lambda x: num_faces[x])(total_errors.test_idx)
# Selection Criteria: lowest overall amount of loss with at least 20 faces

to_render_idx = total_errors[total_errors.faces > 20].sort_values('total_value',ascending=True).test_idx.values[:rows*cols]

print('Rendering')
gts = []
renders = []
with ZipFile(data_path, 'r') as zf:    
    for idx in tqdm(to_render_idx):
        key = index['template'].format(*index['test'][idx])
        with zf.open(key, 'r') as f:
            part_data = f.read().decode('utf-8')
        data = ds_test[idx]
        with torch.no_grad():
            pred = model.grid_enc_dec(data, N)
        V, F, C = preds_to_mesh(pred[0], N)
        opts = PartOptions()
        opts.set_quality = True
        opts.quality = 0.001
        part = Part(part_data, opts)

        n_faces = part.mesh_topology.face_to_topology.max() + 1

        color_pallet = np.ones((n_faces,3)).astype(int)*150
        
        camera_params = get_camera_angle(
            part.mesh.V, part.mesh.F, part.mesh_topology.face_to_topology,
            optimize='seg'
        )

        gt_im = render_segmented_mesh(
            part.mesh.V, part.mesh.F, part.mesh_topology.face_to_topology, color_pallet,
            camera_params=camera_params,
            render_params=RendererParams(imsize,imsize)
            )

        norm_center, norm_scale = get_norm_factors(data.V.numpy())
        
        recon_im = render_segmented_mesh(
            V, F, C, color_pallet, camera_params=camera_params,
            norm_center=norm_center,
            norm_scale=norm_scale,
            render_params=RendererParams(imsize,imsize)
        )

        gts.append(gt_im)
        renders.append(recon_im)

image_rows = []
for r in range(rows):
    curr_row = []
    for c in range(cols):
        idx = r*cols+c
        gt = gts[idx]
        recon = renders[idx]
        curr_row.append(gt)
        curr_row.append(recon)
    curr_row = np.stack(curr_row)
    image_rows.append(curr_row)
image_rows = np.stack(image_rows)

image_grid = grid_images(image_rows)

image = Image.fromarray(image_grid.astype(np.uint8))

image.save(figure_out_path)

Loading Model
Loading Index
Counting Faces
Selecting Parts
Rendering


/tmp/ipykernel_52996/838619694.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors['faces'] = np.vectorize(lambda x: num_faces[x])(errors.test_idx)
/tmp/ipykernel_52996/838619694.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors['total_value'] = errors.value * errors.faces
100%|██████████| 24/24 [03:22<00:00,  8.46s/it]


## Limitations Plot

In [20]:
import altair as alt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import altair_saver

accs = pd.read_parquet(os.path.join(results_dir, 'fusion360seg_reconstruction_vs_accuracy.parquet'))
accs['one'] = 1
iscorr = np.vectorize(lambda x: 'correct' if x == 1 else 'incorrect')
accs['correct'] = iscorr(accs.label)
accs['logmse'] = np.log(accs.mse)

"""
alt.data_transformers.disable_max_rows()

lim_plot = alt.Chart(accs[accs.mse < 1]).mark_bar().encode(
    x=alt.X('logmse',bin=alt.Bin(maxbins=50)),
    y=alt.Y('sum(one)', stack='normalize', axis=alt.Axis(format='%')),
    color=alt.Color('label:N')
).facet(row='train_size')

altair_saver.save(lim_plot, os.path.join(outdir, 'limitations.pdf'))
"""

"\nalt.data_transformers.disable_max_rows()\n\nlim_plot = alt.Chart(accs[accs.mse < 1]).mark_bar().encode(\n    x=alt.X('logmse',bin=alt.Bin(maxbins=50)),\n    y=alt.Y('sum(one)', stack='normalize', axis=alt.Axis(format='%')),\n    color=alt.Color('label:N')\n).facet(row='train_size')\n\naltair_saver.save(lim_plot, os.path.join(outdir, 'limitations.pdf'))\n"

In [80]:
import torch
import pandas as pd
import altair as alt
import altair_saver

mse_losses = torch.load('../../results/f360_recon_mse_losses.pt')
f360_segmentation = pd.read_parquet('../../results/f360_segmentation.parquet')
loss_records = []
for test_idx, losses in enumerate(mse_losses):
    for face_idx, loss in enumerate(losses):
        loss_records.append({
            'test_idx':test_idx,
            'face_idx':face_idx,
            'mse_loss':loss.item()
        })
mse_losses = pd.DataFrame.from_records(loss_records)
f360_segmentation = f360_segmentation.merge(mse_losses, on=['test_idx','face_idx'])
f360_segmentation = f360_segmentation[f360_segmentation.model == 'Ours']
f360_segmentation = f360_segmentation[f360_segmentation.mse_loss < 10]
bins = pd.cut(np.log(f360_segmentation.mse_loss), 10)
bin_left = np.vectorize(lambda x: x.left)(bins)
bin_right = np.vectorize(lambda x: x.right)(bins)
f360_segmentation['bin_left'] = bin_left
f360_segmentation['bin_right'] = bin_right
binned_acc = f360_segmentation.groupby(['train_size','bin_left', 'bin_right']).agg({'accuracy':'mean'}).reset_index()
seg_v_recon = alt.Chart(binned_acc).mark_bar().encode(
    x = alt.X('bin_left', title='Reconstruction log mse (binned)'),
    x2 = alt.X2('bin_right', title=None),
    y = alt.Y('accuracy', title='Segmentation Accuracy')
).properties(title = 'Segmentation vs Reconstruction Accuracy')
altair_saver.save(seg_v_recon, os.path.join(outdir, 'seg_v_recon.pdf'))

## Classification Comparisons

In [ ]:
from plotting import render_segmentation_comparisons_newplotting
from PIL import Image
import numpy as np

f360_comparisons = render_segmentation_comparisons_newplotting(root=repbrep, seg_pred_path = '../../results/f360_segmentation.parquet')
Image.fromarray(f360_comparisons.astype(np.uint8)).save(os.path.join(outdir, 'f360seg-comparisons.png'))

In [ ]:
from plotting import render_segmentation_comparisons_newplotting
from PIL import Image
import numpy as np

mfcad_comparisons = render_segmentation_comparisons_newplotting(
    root=repbrep,
    dataset_name='mfcad',
    camera_name='mfcad',
    dataset='MFCAD',
    max_labels=16,
    seg_pred_path = '../../results/mfcad_segmentation.parquet'
)
Image.fromarray(mfcad_comparisons.astype(np.uint8)).save(os.path.join(outdir, 'mfcad-comparison.png'))

## Segmentation Gallery

In [101]:
import numpy as np
from rendering import render_segmented_mesh, grid_images, RendererParams
from util import ZippedDataset
from automate import Part
from PIL import Image
from matplotlib import pyplot as plt


# Counted from image
rowcol = np.array([[0,7],
[1,11],
[2,9],
[4,3],
[7,18],
[10,2],
[13,10],
[15,14],
[18,3],
[18,15]])
test_indices = rowcol.dot(np.array([[20],[1]])).flatten()

ds = ZippedDataset(os.path.join(repbrep, 'datasets', 'fusion360seg'))

paths = [ds.index['template'].format(*ds.index['test'][i]) for i in test_indices]
parts = [Part(ds.zip.open(p,'r').read().decode('utf-8')) for p in paths]
seg_preds = pd.read_parquet(os.path.join(results_dir, 'f360_segmentation.parquet'))

seg_preds = seg_preds[
    (seg_preds.dataset == 'Fusion360Seg') &
    (seg_preds.model == 'Ours') &
    (seg_preds.seed == 0) &
    (seg_preds.train_size == 23266)
]

preds_and_labels = seg_preds.sort_values(['test_idx','face_idx']).groupby('test_idx').agg({'label':list,'prediction':list}).reset_index()

preds = preds_and_labels.prediction.values[test_indices]
labels = preds_and_labels.label.values[test_indices]

color_pallet = plt.get_cmap('tab10')(np.arange(8))[:,:3]

renders = []
for part,label,pred in zip(parts,labels,preds):
    V = part.mesh.V
    F = part.mesh.F
    F_id = part.mesh_topology.face_to_topology
    l = np.array(label)
    p = np.array(pred)
    l = color_pallet[l]
    p = color_pallet[p]
    pred_im = render_segmented_mesh(V,F,F_id,p,render_params=RendererParams(800,800),camera_opt='seg')
    label_im = render_segmented_mesh(V,F,F_id,l,render_params=RendererParams(800,800),camera_opt='seg')
    renders.append(grid_images(np.stack([np.stack([label_im, pred_im])])))
image_grid = grid_images(np.stack(renders).reshape((5,2,800,1600,4)))
Image.fromarray(image_grid.astype(np.uint8)).save(os.path.join(outdir, 'segmentation_gallery.png'))

In [103]:
seg_preds

,dataset,model,train_size,seed,test_idx,face_idx,prediction,label,accuracy
6615750,Fusion360Seg,Ours,23266,0,0,0,6,6,True
6615751,Fusion360Seg,Ours,23266,0,0,1,6,6,True
6615752,Fusion360Seg,Ours,23266,0,0,2,6,6,True
6615753,Fusion360Seg,Ours,23266,0,0,3,6,6,True
6615754,Fusion360Seg,Ours,23266,0,0,4,6,6,True
...,...,...,...,...,...,...,...,...,...
6668671,Fusion360Seg,Ours,23266,0,4183,11,6,6,True
6668672,Fusion360Seg,Ours,23266,0,4183,12,6,6,True
6668673,Fusion360Seg,Ours,23266,0,4183,13,6,6,True
6668674,Fusion360Seg,Ours,23266,0,4183,14,6,6,True


## Dataset Gallery

## BRep Hierarchy

In [ ]:
from automate import Part
from rendering import render_segmented_mesh, RendererParams
from util import arr2im

part = Part(os.path.join(repbrep, 'datasets', 'figure_parts', 'philipsbolt.step'))
V = part.mesh.V
F = part.mesh.F
F_id = part.mesh_topology.face_to_topology

renders = []
colors = np.stack([np.array([50,50,50,50])]*len(F_id))
renders.append(np.stack([render_segmented_mesh(V,F,F_id,colors,transparent_bg=False)]))
for i in tqdm(range(5)):
    colors = np.stack([np.array([50,50,50,50])]*len(F_id))
    colors[i] = [250,50,50,255]
    renders.append(np.stack([render_segmented_mesh(V,F,F_id,colors,transparent_bg=False)]))
arr2im(grid_images(np.stack(renders))).save(os.path.join(outdir, 'transparent_bolts.png'))

## Time Comparisons

In [ ]:
import pandas as pd
import altair as alt
import altair_saver

edf = pd.read_parquet(os.path.join(results_dir, 'tf_events.parquet'))

fv_edf = edf[
    edf.metric.str.contains('val') & 
    edf.metric.str.contains('loss') & 
    (~edf.metric.str.contains('step')) &
    (edf.dataset == 'f360seg')]

def get_stop_times(x):
    min_time = x.rel_time.values[x.value.values.argmin()]
    max_time = x.rel_time.values[x.value.values.argmax()]
    return pd.Series([min_time, max_time], index=['min_time','max_time'])
stopping_times = fv_edf.groupby(
    ['model','train_size','seed','metric']
    ).apply(get_stop_times).reset_index()

model_names = {
    'ours':'Ours',
    'uvnet':'UV-Net',
    'brepnet':'BRepNet'
}
rename_models = np.vectorize(lambda x: model_names[x])
stopping_times.model = rename_models(stopping_times.model)

stopping_times.min_time = stopping_times.min_time / 60

line = alt.Chart(stopping_times)\
    .mark_line()\
    .encode(
        x=alt.X('train_size', scale=alt.Scale(type='linear'),axis=alt.Axis(title='Training Set Size')),
        y=alt.Y('mean(min_time)', scale=alt.Scale(type='linear'),axis=alt.Axis(title='Training Time (minutes)')),
        color=alt.Color('model',sort=['Ours','UV-Net','BRepNet'],legend=alt.Legend(title='Model'))
    )
band = alt.Chart(stopping_times).mark_errorband(extent='ci').encode(
        x=alt.X('train_size', scale=alt.Scale(type='linear'),axis=alt.Axis(title='Training Set Size')),
        y=alt.Y('min_time', scale=alt.Scale(type='linear'),axis=alt.Axis(title='Training Time (minutes)')),
        color=alt.Color('model',sort=['Ours','UV-Net','BRepNet'],legend=alt.Legend(title='Model'))
    )
chart = (band + line).properties(title='Fusion 360 Segmentation Training Time vs Training Size')

altair_saver.save(chart, os.path.join(outdir, 'f360seg-time.pdf'))

In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import altair_saver

edf = pd.read_parquet(os.path.join(results_dir, 'tf_events.parquet'))

fv_edf = edf[
    edf.metric.str.contains('val') & 
    edf.metric.str.contains('loss') & 
    (~edf.metric.str.contains('step')) &
    (edf.dataset == 'fabwave')]

def get_stop_times(x):
    min_time = x.rel_time.values[x.value.values.argmin()]
    max_time = x.rel_time.values[x.value.values.argmax()]
    return pd.Series([min_time, max_time], index=['min_time','max_time'])
stopping_times = fv_edf.groupby(
    ['model','train_size','seed','metric']
    ).apply(get_stop_times).reset_index()

model_names = {
    'ours':'Ours',
    'uvnet':'UV-Net',
    'brepnet':'BRepNet'
}
rename_models = np.vectorize(lambda x: model_names[x])
stopping_times.model = rename_models(stopping_times.model)

stopping_times.min_time = stopping_times.min_time / 60

line = alt.Chart(stopping_times)\
    .mark_line()\
    .encode(
        x=alt.X('train_size', scale=alt.Scale(type='linear'),axis=alt.Axis(title='Training Set Size')),
        y=alt.Y('mean(min_time)', scale=alt.Scale(type='linear'),axis=alt.Axis(title='Training Time (minutes)')),
        color=alt.Color('model',sort=['Ours','UV-Net','BRepNet'],legend=alt.Legend(title='Model'))
    )
band = alt.Chart(stopping_times).mark_errorband(extent='ci').encode(
        x=alt.X('train_size', scale=alt.Scale(type='linear'),axis=alt.Axis(title='Training Set Size')),
        y=alt.Y('min_time', scale=alt.Scale(type='linear'),axis=alt.Axis(title='Training Time (minutes)')),
        color=alt.Color('model',sort=['Ours','UV-Net','BRepNet'],legend=alt.Legend(title='Model'))
    )
chart = (band + line).properties(title='Fusion 360 Segmentation Training Time vs Training Size')
chart
#altair_saver.save(chart, os.path.join(outdir, 'f360seg-time.pdf'))

## Clipping Plane

In [ ]:
from hybridbrep import HPart
import os
from matplotlib import pyplot as plt
import numpy as np
import torch
N = 1000#00
part_path = os.path.join(repbrep, 'datasets/figure_parts/philipsbolt.step')
data = HPart(part_path, N, N, True).data
u = data.surface_coords[0,:,0].numpy()
v = data.surface_coords[0,:,1].numpy()
#v = v.max() - v
m = data.surface_samples[0,:,-1].T.numpy()

((u_min, v_min), (u_max, v_max)) = data.surface_bounds[0].numpy()
u_prime = ((u *(u_max - u_min) + u_min) - (2*np.pi))
v_prime = (v * (v_max - v_min) + v_min)

plt.scatter(u_prime,v_prime,c=(m <= 0))
plt.colorbar()
plt.gca().set_aspect('equal')


In [ ]:
plt.scatter(u_prime,v_prime,c=(m),s=.03,cmap='tab20')
plt.colorbar()
plt.xlim(0,2*np.pi)
plt.ylim(-np.pi/2, np.pi/2)
plt.gca().set_aspect('equal')


In [ ]:
plt.scatter(u,v,c=(m<=0),s=.2, cmap='tab20')
plt.colorbar()
#plt.xlim(0,2*np.pi)
#plt.ylim(-np.pi/2, np.pi/2)
plt.gca().set_aspect('equal')

In [ ]:
import meshplot as mp

In [ ]:
xyz = data.surface_samples[0,:,:3].numpy()

In [ ]:
mp.plot(xyz, c=m, shading={'point_size':.03})

In [ ]:
part_path_step = os.path.join(repbrep, 'datasets/frame_guide/fg1.step')
part_path_x_t = os.path.join(repbrep, 'datasets/frame_guide/fg1.x_t')

for frac in np.linspace(0,1.0,6):

    data = HPart(part_path_x_t, 500, 5000, True, frac).data

    u = data.surface_coords[11,:,0].numpy()
    v = data.surface_coords[11,:,1].numpy()
    #v = v.max() - v
    m = data.surface_samples[11,:,-1].T.numpy()

    plt.figure()
    plt.scatter(u,v,c=m)
    plt.colorbar()
    plt.title(f'Sampling with {frac} sorted')